In [1]:
import os 

lang = "ja"
train_data_root = "/data/xzhao/experiments/med-eval/dataset/losses/train_samples"
data_path = os.path.join(train_data_root, f"{lang}_10k.jsonl")

In [14]:
import nltk
import hanlp
split_sent = hanlp.load(hanlp.pretrained.eos.UD_CTB_EOS_MUL)

def split_document(text, lang='en'):
    if lang == 'en' or lang == 'en_jstage':
        split_sentences = nltk.tokenize.sent_tokenize(text)
    elif lang == 'ja' or lang == 'zh':
        split_sentences = split_sent(text)

    sentences = []
    start_ids, end_ids = [], []
    for sent in split_sentences:
        start_ids.append(text.index(sent, start_ids[-1] if start_ids else 0))

    for idx, sent in enumerate(split_sentences[1:]):
        end_ids.append(text.index(sent, start_ids[idx + 1]))
    end_ids += [len(text)]

    sent_pos = [(start, end) for start, end in zip(start_ids, end_ids)]
    for start, end in sent_pos:
        if end == -1:
            sentences.append(text[start:])
        else:
            sentences.append(text[start:end])
    assert ''.join(sentences) == text, "Reconstructed document does not match the original."
    return sentences


In [16]:
from tqdm import tqdm
from utils import load_jsonl_iteratively

all_sents = []
for item in tqdm(load_jsonl_iteratively(data_path)):
    text = item["text"]
    split_sentences = split_document(text, lang=lang)
    if len(split_sentences) > 3:
        all_sents.append({'text': text, 'sentences': split_sentences})    
len(all_sents)

5846it [04:25, 22.02it/s] 


KeyboardInterrupt: 

In [30]:
import random

from utils import dump_jsonl 
random.seed(42)
random.shuffle(all_sents)

dump_root = "/home/xzhao/workspace/med-eval/dataset/losses/train_samples"
dump_path = os.path.join(dump_root, f"random_{lang}_2k.jsonl")
dump_jsonl(all_sents[:2000], dump_path)
